Project 1 - Terrence Cummings
- Crime, Schools, and Home Values in Minneapolis

Step 1 - Housing data
- Read in housing data from "Open Minneapolis" website using APIs (currently hit 2000 limit. Need to fix.)
- Store in dataframe
- Cleanup data
- Convert x, y map coordinates to lat/lng

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import time
from scipy.stats import linregress
import json
from pprint import pprint
from datetime import datetime


In [ ]:
#Request home sales data from Open Minnesota API

home_sales_geoserv = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Property_Sales_2015_to_2019/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'

home_sales_data = requests.get(house_sales_geoserv).json()


In [ ]:
#Set the number of home sales in the data for looping
num_houses=len(home_sales_data['features'])

#Initalize lists of key data
sale_id =[]
sale_date = []
formatted_address = []
land_sale = []
community_cd = []
community_desc = []
nbhd_cd = []
nbhd_desc = []
ward = []
proptype_cd = []
proptype_desc = []
adj_sale_price = []
gross_sale_price = []
downpayment = []
x_coord = []
y_coord = []

#Fill lists of key data
for house in range(0, num_houses):
    sale_id.append(home_sales_data['features'][house]['attributes']['SALE_ID'])
    sale_date.append(home_sales_data['features'][house]['attributes']['SALE_DATE'])
    formatted_address.append(home_sales_data['features'][house]['attributes']['FORMATED_ADDRESS'])
    land_sale.append(home_sales_data['features'][house]['attributes']['LAND_SALE'])
    community_cd.append(home_sales_data['features'][house]['attributes']['COMMUNITY_CD'])
    community_desc.append(home_sales_data['features'][house]['attributes']['COMMUNITY_DESC'])
    nbhd_cd.append(home_sales_data['features'][house]['attributes']['NBHD_CD'])
    nbhd_desc.append(home_sales_data['features'][house]['attributes']['NBHD_DESC'])
    ward.append(home_sales_data['features'][house]['attributes']['WARD'])
    proptype_cd.append(home_sales_data['features'][house]['attributes']['PROPTYPE_CD'])
    proptype_desc.append(home_sales_data['features'][house]['attributes']['PROPTYPE_DESC'])
    adj_sale_price.append(home_sales_data['features'][house]['attributes']['ADJ_SALE_PRICE'])
    gross_sale_price.append(home_sales_data['features'][house]['attributes']['GROSS_SALE_PRICE'])
    downpayment.append(home_sales_data['features'][house]['attributes']['DOWNPAYMENT'])
    x_coord.append(home_sales_data['features'][house]['attributes']['X'])
    y_coord.append(home_sales_data['features'][house]['attributes']['Y'])


In [214]:
#Create df of all MSP home sales data
msp_home_sales_df = pd.DataFrame(zip(sale_id, sale_date, formatted_address, land_sale, community_cd, community_desc, nbhd_cd, nbhd_desc, ward, proptype_cd, proptype_desc, adj_sale_price, gross_sale_price, downpayment, x_coord, y_coord), columns = ['sale_id', 'sale_date', 'formatted_address', 'land_sale', 'community_cd', 'community_desc', 'nbhd_cd', 'nbhd_desc', 'ward', 'proptype_cd', 'proptype_desc', 'adj_sale_price', 'gross_sale_price', 'downpayment', 'x_coord', 'y_coord'])

msp_home_sales_df.set_index('sale_id', inplace=True)

#Create a df of the community codes and descriptions
community_df = pd.DataFrame(zip(community_cd, community_desc), columns=['community_cd', 'community_desc'])
community_df = community_df.drop_duplicates()
community_df = community_df.sort_values(['community_cd'])
community_df.set_index('community_cd', inplace = True)

#Create a df of the neighborhood codes and descriptions
nbhd_df = pd.DataFrame(zip(nbhd_cd, nbhd_desc), columns=['nbhd_cd', 'nbhd_desc'])
nbhd_df = nbhd_df.drop_duplicates()
nbhd_df = nbhd_df.sort_values(['nbhd_cd'])
nbhd_df.set_index('nbhd_cd', inplace = True)

#Create a df of the property type codes and descriptions
proptype_df = pd.DataFrame(zip(proptype_cd, proptype_desc), columns=['proptype_cd', 'proptype_desc'])
proptype_df = proptype_df.drop_duplicates()
proptype_df = proptype_df.sort_values(['proptype_cd'])
proptype_df.set_index('proptype_cd', inplace = True)

msp_home_sales_df['proptype_cd'].value_counts()

R     1308
X      331
DB     154
C       73
A       42
LR      30
Y       24
LC       7
I        6
XM       5
TP       5
RZ       4
LA       4
RM       3
LI       2
NH       2
Name: proptype_cd, dtype: int64

In [222]:
#Clean up df by eliminating 0 sale price
clean_msp_home_sales_df = msp_home_sales_df.loc[msp_home_sales_df['adj_sale_price']>10000]

#Clean up by eliminating non-residential and odd property types
proptype_allowed = ['R', 'X', 'DB', 'Y', 'RZ' 'RM']
clean_msp_home_sales_df = clean_msp_home_sales_df[clean_msp_home_sales_df['proptype_cd'].isin(proptype_allowed)]

#Clean up by eliminating land-only sales
clean_msp_home_sales_df = clean_msp_home_sales_df[clean_msp_home_sales_df['land_sale']=='NO']

clean_msp_home_sales_df

,sale_date,formatted_address,land_sale,community_cd,community_desc,nbhd_cd,nbhd_desc,ward,proptype_cd,proptype_desc,adj_sale_price,gross_sale_price,downpayment,x_coord,y_coord
sale_id,,,,,,,,,,,,,,,
304279,1420783200000,3542 Valley St,NO,2,NORTHEAST,11,COLUMBIA,1,R,RESIDENTIAL,145985,150500,7525,533346.190261,188201.516808
303578,1420783200000,301 Oak Grove St #506,NO,4,CENTRAL,31,LORING PARK,7,X,CONDOMINIUM,241000,241000,0,525967.100955,163990.980501
304190,1420783200000,3136 West Bde Maka Ska Blvd W #206,NO,6,CALHOUN-ISLE,49,WEST CALHOUN,13,X,CONDOMINIUM,20000,20000,0,516063.000000,156554.000000
303562,1420783200000,730 4TH St N #601,NO,4,CENTRAL,86,NORTHLOOP,3,X,CONDOMINIUM,320000,320000,0,526709.800000,171382.570000
304311,1420783200000,3439 Polk St,NO,2,NORTHEAST,12,WAITE PARK,1,R,RESIDENTIAL,153500,160000,18741,535920.652943,187323.980831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305657,1429246800000,2809 35TH Ave S,NO,8,LONGFELLOW,61,LONGFELLOW,2,R,RESIDENTIAL,183000,185000,3351,541953.620586,158544.594286
306385,1429246800000,3843 5TH Ave S,NO,7,POWDERHORN,57,BRYANT,8,DB,DOUBLE BUNGALOW,82577,82577,0,529743.563171,151593.562848
305988,1429246800000,5717 Emerson Ave S,NO,9,SOUTHWEST,67,KENNY,13,R,RESIDENTIAL,250000,255500,0,523075.125622,139390.137337
